In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
'''Used to help format and organize the messy web by fixing bad HTML and presenting us with
easily-traversible Python objects repreenting XML structures.'''
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json

In [3]:
class TwitterClient(object):
    def __init__(self):
        #Twitter Credentials
        with open('twitter_cred.json') as secret_info:
            info = json.load(secret_info)
            consumer_key = info['CONSUMER_KEY']
            consumer_secret = info['CONSUMER_SECRET']
            access_key = info['ACCESS_KEY']
            access_secret = info['ACCESS_SECRET']
            
            try:

                # Creating the API endpoint
                auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
                #setting the access token and secret
                auth.set_access_token(access_key,access_secret)
                self.api = tweepy.API(auth,wait_on_rate_limit=True,
                wait_on_rate_limit_notify=True)
            except tweepy.TweepError as e:
                print(f"Error: Twitter Authentication Failed - \n{str(e)}")
    
    #Function used to fetch tweets
    def get_tweets(self,query,maxTweets = 1000):
        tweets = []
        sinceId = []
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100
        
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query,count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query,count=tweetsPerQry,since_id = sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query,count=tweetsPerQry,max_id=str(max_id-1))
                    else:
                        new_tweets = self.api.search(q=query,count=tweetsPerQry,max_id=str(max_id-1),since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                
                for tweet in new_tweets:
                    parsed_tweet = {}
                    parsed_tweet['tweets'] = tweet.text
                    
                    #Appending parsed tweets to tweets list
                    if tweet.retweet_count > 0:
                        #if the tweet has retweets, ensure that it is appended only once
                        if parsed_tweet not in tweets:
                            tweets.append(parsed_tweet)
                    else:
                        tweets.append(parsed_tweet)
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id
            except tweepy.TweepError as e:
                print("Tweepy error:" + str(e))
                break
        t = pd.DataFrame(tweets)
        t.columns = ['List of Tweets']
        with open('tweets_pull1.csv','a+') as f:
            t.to_csv(f,header=['List of Tweets'])
        return t
                        
                        
                
        
        

In [4]:
selenium_client = TwitterClient()
tweets_df = selenium_client.get_tweets('drought')
tweets_df

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets


,List of Tweets
0,Barnaby Joyce yet to produce a single report f...
1,RT @iMusing: Show me a straw man argument and ...
2,The thing you missed here PM is that as a govt...
3,RT @Phil_B7: @lynlinking @blanketcrap $100m fo...
4,I think I'll buy a property in a drought stric...
...,...
450,@9NewsAUS @Fi_Willan There is no drought lifel...
451,Hypocrite @ScottMorrisonMP ropes in primary sc...
452,The news media is hiding a potential disastrou...
453,RT @capitalweather: DC hit at least 90 degrees...
